In [2]:
from gurobipy import * 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import copy
import geopandas as gpd 
%matplotlib inline 

In [1]:
import scipy.sparse as sp
from itertools import product

# Other constants 

In [101]:
WAITING_TIME_COST = 1 # $ dollar 
DENIED_COST = 5 

In [17]:
start = 10
end = 15
time_periods = list(range(start, end))
time_periods

[10, 11, 12, 13, 14]

# Read network's setup (e.g., travel time/cost)

In [3]:
dist_mat = pd.read_csv('/Users/peyman/Dropbox (MIT)/Projects/RL_ridehailing/env/Data/dist_mat_2.csv')

# Zones

In [4]:
zone_ids_file = pd.read_csv("/Users/peyman/Dropbox (MIT)/Projects/RL_ridehailing/env/Data/zones_w_neighbors.csv")
zone_ids = zone_ids_file.LocationID.values
zone_ids

array([  4,  12,  13,  24,  41,  42,  43,  45,  48,  50,  68,  74,  75,
        79,  87,  88,  90, 100, 107, 113, 114, 116, 120, 125, 127, 128,
       137, 140, 141, 142, 143, 144, 148, 151, 152, 158, 161, 162, 163,
       164, 166, 170, 186, 194, 209, 211, 224, 229, 230, 231, 232, 233,
       234, 236, 237, 238, 239, 243, 244, 246, 249, 261, 262, 263])

In [63]:
64*64

4096

In [5]:
ZONE_IDS = list(set(zone_ids).intersection(dist_mat.PULocationID.unique()))

In [6]:
len(ZONE_IDS)

64

### exclude from dist matrix those ODs that are not in zone ids

In [96]:
dist_mat = dist_mat[(dist_mat.PULocationID.isin(ZONE_IDS) & dist_mat.DOLocationID.isin(ZONE_IDS) )]

### rebalancing cost matrix 

In [19]:
dist_mat.head()

,PULocationID,DOLocationID,trip_distance_meter
0,4,4,1124.0
1,4,12,5802.0
2,4,13,6471.0
3,4,24,11505.0
4,4,41,11405.0


In [97]:
cost_mat = tupledict()
for _, i in dist_mat.iterrows():
    cost_mat[(int(i.PULocationID), int(i.DOLocationID))] = int(i.trip_distance_meter)
    

### waiting cost

In [111]:
waiting_cost_mat = tupledict()
for _, i in dist_mat.iterrows():
    waiting_cost_mat[(int(i.PULocationID), int(i.DOLocationID))] = WAITING_TIME_COST
    

### denied cost

In [102]:
denied_cost_mat = tupledict()
for _, i in dist_mat.iterrows():
    denied_cost_mat[(int(i.PULocationID), int(i.DOLocationID))] = DENIED_COST
    

### od pairs

In [98]:
od_pairs = (dist_mat[["PULocationID", "DOLocationID"]].values)

In [99]:
od_pairs

array([[  4,   4],
       [  4,  12],
       [  4,  13],
       ...,
       [263, 261],
       [263, 262],
       [263, 263]])

In [100]:
od_pairs = [tuple(i) for i in od_pairs]
len(od_pairs)

4096

In [68]:
od_pairs

[(4, 4),
 (4, 12),
 (4, 13),
 (4, 24),
 (4, 41),
 (4, 42),
 (4, 43),
 (4, 45),
 (4, 48),
 (4, 50),
 (4, 68),
 (4, 74),
 (4, 75),
 (4, 79),
 (4, 87),
 (4, 88),
 (4, 90),
 (4, 100),
 (4, 105),
 (4, 107),
 (4, 113),
 (4, 114),
 (4, 116),
 (4, 120),
 (4, 125),
 (4, 127),
 (4, 128),
 (4, 137),
 (4, 140),
 (4, 141),
 (4, 142),
 (4, 143),
 (4, 144),
 (4, 148),
 (4, 151),
 (4, 152),
 (4, 153),
 (4, 158),
 (4, 161),
 (4, 162),
 (4, 163),
 (4, 164),
 (4, 166),
 (4, 170),
 (4, 186),
 (4, 194),
 (4, 202),
 (4, 209),
 (4, 211),
 (4, 224),
 (4, 229),
 (4, 230),
 (4, 231),
 (4, 232),
 (4, 233),
 (4, 234),
 (4, 236),
 (4, 237),
 (4, 238),
 (4, 239),
 (4, 243),
 (4, 244),
 (4, 246),
 (4, 249),
 (4, 261),
 (4, 262),
 (4, 263),
 (12, 4),
 (12, 12),
 (12, 13),
 (12, 24),
 (12, 41),
 (12, 42),
 (12, 43),
 (12, 45),
 (12, 48),
 (12, 50),
 (12, 68),
 (12, 74),
 (12, 75),
 (12, 79),
 (12, 87),
 (12, 88),
 (12, 90),
 (12, 100),
 (12, 105),
 (12, 107),
 (12, 113),
 (12, 114),
 (12, 116),
 (12, 120),
 (12, 125),

# Read OD 

## Select t 

In [123]:
demand = pd.read_csv("/Users/peyman/Dropbox (MIT)/Projects/RL_ridehailing/env/Data/Daily_demand/demand_for_day_2.csv")
demand.shape

(197746, 10)

In [124]:
demand = demand[demand.PULocationID.isin(ZONE_IDS) & demand.DOLocationID.isin(ZONE_IDS)]
demand.shape

(197679, 10)

In [127]:
od = tupledict()

In [126]:
demand = demand.set_index(["PULocationID","DOLocationID", "time_of_day_index_15m"])

In [139]:
# https://stackoverflow.com/questions/54307300/what-causes-indexing-past-lexsort-depth-warning-in-pandas
demand = demand.sort_index()

In [140]:
demand

tpep_dropoff_datetime  \
PULocationID DOLocationID time_of_day_index_15m                         
4            4            21                      2018-01-02 05:42:16   
                          30                      2018-01-02 07:38:14   
                          32                      2018-01-02 08:13:27   
                          48                      2018-01-02 12:14:44   
                          48                      2018-01-02 12:10:53   
                          52                      2018-01-02 13:17:07   
                          55                      2018-01-02 14:01:06   
                          55                      2018-01-02 13:53:23   
                          58                      2018-01-02 14:46:47   
                          61                      2018-01-02 15:37:43   
                          64                      2018-01-02 16:16:01   
                          66                      2018-01-02 16:45:43   
                          67                      2018-01-02 16:48:17   
                          71                      2018-01-02 17:50:11   
                          71                      2018-01-02 17:47:16   
                          74                      2018-01-02 18:32:21   
                          74                      2018-01-02 18:38:58   
                          80                      2018-01-02 20:14:02   
                          83                      2018-01-02 20:53:34   
                          86                      2018-01-02 21:40:53   
             13           30                      2018-01-02 07:57:06   
                          37                      2018-01-02 09:40:35   
                          46                      2018-01-02 11:47:52   
                          56                      2018-01-02 14:24:21   
                          84                      2018-01-02 21:16:56   
             41           65                      2018-01-02 16:59:04   
             43           46                      2018-01-02 11:49:30   
             45           33                      2018-01-02 08:26:12   
                          34                      2018-01-02 08:47:18   
                          43                      2018-01-02 11:08:27   
...                                                               ...   
263          263          82                      2018-01-02 20:44:14   
                          83                      2018-01-02 20:54:17   
                          83                      2018-01-02 20:54:57   
                          83                      2018-01-02 20:51:42   
                          83                      2018-01-02 20:51:49   
                          84                      2018-01-02 21:07:37   
                          84                      2018-01-02 21:14:08   
                          84                      2018-01-02 21:26:17   
                          85                      2018-01-02 21:33:09   
                          85                      2018-01-02 21:23:24   
                          85                      2018-01-02 21:24:03   
                          85                      2018-01-02 21:26:02   
                          85                      2018-01-02 21:27:27   
                          85                      2018-01-02 21:30:00   
                          86                      2018-01-02 21:34:43   
                          86                      2018-01-02 21:36:04   
                          87                      2018-01-02 21:57:01   
                          88                      2018-01-02 22:20:32   
                          88                      2018-01-02 22:07:38   
                          89                      2018-01-02 22:19:41   
                          89                      2018-01-02 22:25:01   
                          89                      2018-01-02 22:23:15   
                          90     

In [141]:
demand.loc[90, 114, 40]["passenger_count"].sum()

6

## Populate OD matrix

In [143]:
for t in time_periods: 
    for o in ZONE_IDS:
        for d in ZONE_IDS:
            try:
                od[(t,o,d)] = demand.loc[o,d,t]["passenger_count"].sum()
            except:
                od[(t,o,d)] = 0 

In [144]:
od

{(10, 128, 128): 0,
 (10, 128, 4): 0,
 (10, 128, 261): 0,
 (10, 128, 262): 0,
 (10, 128, 263): 0,
 (10, 128, 137): 0,
 (10, 128, 12): 0,
 (10, 128, 13): 0,
 (10, 128, 140): 0,
 (10, 128, 141): 0,
 (10, 128, 142): 0,
 (10, 128, 143): 0,
 (10, 128, 144): 0,
 (10, 128, 148): 0,
 (10, 128, 151): 0,
 (10, 128, 24): 0,
 (10, 128, 152): 0,
 (10, 128, 158): 0,
 (10, 128, 161): 0,
 (10, 128, 162): 0,
 (10, 128, 163): 0,
 (10, 128, 164): 0,
 (10, 128, 166): 0,
 (10, 128, 41): 0,
 (10, 128, 42): 0,
 (10, 128, 43): 0,
 (10, 128, 170): 0,
 (10, 128, 45): 0,
 (10, 128, 48): 0,
 (10, 128, 50): 0,
 (10, 128, 186): 0,
 (10, 128, 194): 0,
 (10, 128, 68): 0,
 (10, 128, 74): 0,
 (10, 128, 75): 0,
 (10, 128, 79): 0,
 (10, 128, 209): 0,
 (10, 128, 211): 0,
 (10, 128, 87): 0,
 (10, 128, 88): 0,
 (10, 128, 90): 0,
 (10, 128, 224): 0,
 (10, 128, 100): 0,
 (10, 128, 229): 0,
 (10, 128, 230): 0,
 (10, 128, 231): 0,
 (10, 128, 232): 0,
 (10, 128, 233): 0,
 (10, 128, 234): 0,
 (10, 128, 107): 0,
 (10, 128, 236): 0

In [145]:
ODs = od.keys()

# Read or generate vehicle locations (at time t)

In [146]:
veh_loc = tupledict()

In [147]:
for o in ZONE_IDS:
    for d in ZONE_IDS:
        veh_loc[(o,d)] = np.random.randint(0,10)

In [63]:
veh_loc.sum('*')

<gurobi.LinExpr: 18513.0>

In [65]:
veh_loc

{(128, 128): 7,
 (128, 4): 6,
 (128, 261): 2,
 (128, 262): 5,
 (128, 263): 7,
 (128, 137): 8,
 (128, 12): 9,
 (128, 13): 5,
 (128, 140): 6,
 (128, 141): 5,
 (128, 142): 1,
 (128, 143): 1,
 (128, 144): 2,
 (128, 148): 2,
 (128, 151): 0,
 (128, 24): 3,
 (128, 152): 4,
 (128, 158): 3,
 (128, 161): 3,
 (128, 162): 6,
 (128, 163): 2,
 (128, 164): 3,
 (128, 166): 7,
 (128, 41): 3,
 (128, 42): 6,
 (128, 43): 8,
 (128, 170): 5,
 (128, 45): 8,
 (128, 48): 1,
 (128, 50): 4,
 (128, 186): 0,
 (128, 194): 7,
 (128, 68): 7,
 (128, 74): 1,
 (128, 75): 7,
 (128, 79): 1,
 (128, 209): 5,
 (128, 211): 2,
 (128, 87): 1,
 (128, 88): 9,
 (128, 90): 2,
 (128, 224): 9,
 (128, 100): 1,
 (128, 229): 2,
 (128, 230): 6,
 (128, 231): 9,
 (128, 232): 0,
 (128, 233): 8,
 (128, 234): 0,
 (128, 107): 7,
 (128, 236): 1,
 (128, 237): 5,
 (128, 238): 8,
 (128, 239): 2,
 (128, 113): 6,
 (128, 114): 7,
 (128, 243): 0,
 (128, 116): 8,
 (128, 244): 0,
 (128, 246): 4,
 (128, 120): 3,
 (128, 249): 7,
 (128, 125): 9,
 (128, 127

# Rebalancing model 

In [168]:
m = Model("rebal")

In [169]:
serving = m.addVars(time_periods, od_pairs, name="serving",vtype=GRB.INTEGER)
rebal = m.addVars(time_periods, od_pairs, name="rebalancing",vtype=GRB.INTEGER)
waiting = m.addVars(time_periods, od_pairs, name="waiting",vtype=GRB.INTEGER )
denied = m.addVars(time_periods, od_pairs, name="denied",vtype=GRB.INTEGER )

In [170]:
m.update()
m

<gurobi.Model MIP instance rebal: 0 constrs, 81920 vars, No parameter changes>

In [108]:
m.addConstrs((serving[time_period, o, d]  + denied[time_period, o, d] - waiting[time_period, o, d] == od[time_period, o, d] for o, d in od_pairs for time_period in time_periods)

gurobipy.tupledict

In [ ]:
# m.addConstrs((quicksum(waiting[t,o,d]) ==  for t in time_periods))

In [112]:
obj = quicksum(
    rebal[time_period, o, d] * cost_mat[( o, d)] - 
    waiting[time_period, o, d] * waiting_cost_mat[( o, d)] -
    denied[time_period, o, d] * denied_cost_mat[( o, d)] 
# 	  storeCost * held[time_period, product]  
	  for time_period in time_periods for o, d in od_pairs)

In [113]:
m.setObjective(obj, GRB.MAXIMIZE)

In [157]:
m

<gurobi.Model Continuous instance rebal: 0 constrs, 0 vars, No parameter changes>

In [163]:
m.update()

In [164]:
m

<gurobi.Model Continuous instance rebal: 0 constrs, 61440 vars, No parameter changes>

In [85]:
cost_mat

{(4, 4): 1124,
 (4, 12): 5802,
 (4, 13): 6471,
 (4, 24): 11505,
 (4, 41): 11405,
 (4, 42): 13361,
 (4, 43): 8172,
 (4, 45): 2928,
 (4, 48): 6109,
 (4, 50): 7195,
 (4, 68): 4160,
 (4, 74): 10350,
 (4, 75): 9005,
 (4, 79): 1195,
 (4, 87): 4771,
 (4, 88): 5162,
 (4, 90): 3298,
 (4, 100): 4809,
 (4, 105): 10000,
 (4, 107): 1947,
 (4, 113): 2063,
 (4, 114): 1786,
 (4, 116): 15059,
 (4, 120): 18395,
 (4, 125): 2772,
 (4, 127): 18406,
 (4, 128): 10000,
 (4, 137): 2446,
 (4, 140): 5756,
 (4, 141): 6043,
 (4, 142): 7726,
 (4, 143): 8308,
 (4, 144): 2035,
 (4, 148): 1428,
 (4, 151): 11517,
 (4, 152): 15149,
 (4, 153): 22692,
 (4, 158): 3401,
 (4, 161): 5101,
 (4, 162): 4497,
 (4, 163): 6255,
 (4, 164): 3853,
 (4, 166): 12453,
 (4, 170): 3655,
 (4, 186): 4256,
 (4, 194): 10000,
 (4, 202): 9978,
 (4, 209): 4534,
 (4, 211): 2314,
 (4, 224): 1260,
 (4, 229): 4590,
 (4, 230): 5703,
 (4, 231): 3897,
 (4, 232): 1574,
 (4, 233): 3988,
 (4, 234): 2745,
 (4, 236): 8045,
 (4, 237): 6344,
 (4, 238): 10483,
